In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

import string
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv('6.csv')
df.drop("Unnamed: 0", inplace = True, axis = 1)
df.head()

,Text_Data,Category
0,000 0001364 00000000 001 001 01364 INS: 0 0 Pl...,invoices
1,€/@ LES000 Account number 83141467 Page 2 of 4...,invoices
2,i“ CITY OF SEATTLE Seattle Public Utilities Bi...,invoices
3,account number due date amount due 39779 -4...,invoices
4,"account r,umbe'r ouf oatł amount due 39782 ...",invoices


In [3]:
df = df.dropna()

In [4]:

df['Text_Data'] = df['Text_Data'].apply(lambda x: " ".join(x.lower() for x in x.split())) # lower case conversion
df['Text_Data'] = df['Text_Data'].str.replace('[^\w\s]','') # getting rid of special characters
df['Text_Data'] = df['Text_Data'].str.replace('\d+', '') # removing numeric values from between the words
df['Text_Data'] = df['Text_Data'].apply(lambda x: x.translate(string.digits)) # removing numerical numbers
stop = stopwords.words('english')
df['Text_Data'] = df['Text_Data'].apply(lambda x: " ".join(x for x in str(x).split() if x not in stop)) #removing stop words
stemmer = WordNetLemmatizer()
df['Text_Data'] = [stemmer.lemmatize(word) for word in df['Text_Data']] #converting words to their dictionary form
df['Text_Data'] = df['Text_Data'].str.replace('shall', '')

C:\Users\SANYAM\AppData\Local\Temp/ipykernel_13868/3588151574.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text_Data'] = df['Text_Data'].str.replace('[^\w\s]','') # getting rid of special characters
C:\Users\SANYAM\AppData\Local\Temp/ipykernel_13868/3588151574.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text_Data'] = df['Text_Data'].str.replace('\d+', '') # removing numeric values from between the words


In [5]:
df

,Text_Data,Category
0,ins please post payment avidxchange inc reserv...,invoices
1,les account number page philadelphia insurance...,invoices
2,city seattle seattle public utilities bill _ p...,invoices
3,account number due date amount due name parkha...,invoices
4,account rumber ouf oatł amount due meter previ...,invoices
...,...,...
138,ges x old republic onzun x w gowe street suite...,other
139,dear valued customer thank patience aswe conti...,other
140,sammamish plateau water lameness main fax wwws...,other
141,sammamish plateau water oe main fax wwwspwater...,other


In [6]:
# Converting the text data into vectors using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.preprocessing import LabelEncoder
# Generating 1000 features for the input for the model
n=1000
tfidfconverter = TfidfVectorizer(max_features=n, stop_words=stopwords.words('english')) 
X = pd.DataFrame(tfidfconverter.fit_transform(df['Text_Data']).toarray())
print(X[:5])
#X.columns = range(X.shape[1])
labelencoder = LabelEncoder() #Converting the labels to numeric labels
y = labelencoder.fit_transform(df['Category'])


   0         1         2    3    4    5    6    7    8         9    ...  \
0  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  ...   
1  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.094541  ...   
2  0.0  0.027179  0.081152  0.0  0.0  0.0  0.0  0.0  0.0  0.107016  ...   
3  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.139222  ...   
4  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.174903  ...   

        990  991       992  993       994  995  996  997  998  999  
0  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  
1  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  
2  0.085318  0.0  0.000000  0.0  0.074497  0.0  0.0  0.0  0.0  0.0  
3  0.000000  0.0  0.205418  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  
4  0.000000  0.0  0.206452  0.0  0.073054  0.0  0.0  0.0  0.0  0.0  

[5 rows x 1000 columns]


In [7]:
import pickle
pickle.dump(tfidfconverter,open('tf01','wb'))

In [8]:
X.shape, y.shape

((141, 1000), (141,))

In [9]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

In [10]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [11]:
import numpy as np
unique, counts = np.unique(y_resampled, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ 0 42]
 [ 1 42]
 [ 2 42]
 [ 3 42]
 [ 4 42]
 [ 5 42]]


In [12]:

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=10)  

In [13]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
clf_nb = GaussianNB()

# Train the model using the training sets
clf_nb.fit(X_train, y_train)

#Predict Output
y_pred = clf_nb.predict(X_test)
y_pred

array([5, 2, 5, 1, 1, 5, 0, 5, 5, 4, 1, 2, 2, 0, 3, 5, 4, 0, 0, 5, 5, 0,
       1, 1, 5, 4, 3, 3, 5, 1, 2, 2, 2, 5, 0, 3, 1, 3, 2, 0, 3, 5, 4, 2,
       5, 3, 2, 1, 2, 5, 2])

In [14]:
import pickle
pickle.dump(clf_nb, open('nb.pkl','wb'))

In [24]:
#Loading the saved model and a random PDF file to test classification

from pdf2image import convert_from_path
import glob
import pytesseract

def extractTextFromPDF(pdf_location):
    text = ""
    filename = pdf_location.split('/')[::-1][0]
    pdfs = glob.glob(pdf_location)
    for pdf_path in pdfs:
        pages = convert_from_path(pdf_path, 500, poppler_path = r'E:\ocr\poppler-0.68.0\bin')

        for pageNum,imgBlob in enumerate(pages):
            text_per_page = pytesseract.image_to_string(imgBlob, lang='eng')
            text += text_per_page + " "
    
    return (" ".join(text.split())).strip(), filename
        
text_string, filename = extractTextFromPDF("18.pdf")

dfdemo = pd.DataFrame({"Data" : [text_string]})
savedmodel = pickle.load(open('nb.pkl','rb'))
dfdemo

,Data
0,ACCOUNT NUMBER INVOICE DATE INVOICE NUMBER BAR...


In [25]:
# Feature engineering to get the data in right format

dfdemo['Data'] = dfdemo['Data'].apply(lambda x: " ".join(x.lower() for x in x.split())) # lower case conversion
dfdemo['Data'] = dfdemo['Data'].str.replace('[^\w\s]','') # getting rid of special characters
dfdemo['Data'] = dfdemo['Data'].str.replace('\d+', '') # removing numeric values from between the words
dfdemo['Data'] = dfdemo['Data'].apply(lambda x: x.translate(string.digits)) # removing numerical numbers
stop = stopwords.words('english')
dfdemo['Data'] = dfdemo['Data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) #removing stop words
stemmer = WordNetLemmatizer()
dfdemo['Data'] = [stemmer.lemmatize(word) for word in dfdemo['Data']]

C:\Users\Krupal\AppData\Local\Temp\ipykernel_4284\3895234543.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  dfdemo['Data'] = dfdemo['Data'].str.replace('[^\w\s]','') # getting rid of special characters
C:\Users\Krupal\AppData\Local\Temp\ipykernel_4284\3895234543.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dfdemo['Data'] = dfdemo['Data'].str.replace('\d+', '') # removing numeric values from between the words


In [26]:
tfidfconverter = pickle.load(open('tf01','rb'))
inputs = pd.DataFrame(tfidfconverter.transform(dfdemo['Data']).toarray())
inputs

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:

output_category = savedmodel.predict(inputs)
#Comment the next line if you are testing word2vec model as it doesn't require transformation
output_category = (labelencoder.inverse_transform((output_category)))
output_category

array(['invoices'], dtype=object)

In [2]:
import os
old_name = "2.pdf"
new_name = "invoice.pdf"
os.rename(old_name, new_name)